# Catboost 

Data Set link: 
https://drive.google.com/file/d/10BGVH8RujGWPVHLseFrYGUW3cjOav__m/view?usp=drive_link

link https://catboost.ai/

In [3]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv("/Users/olegyarygin/Pandas/Data/uk-used-car/bmw.csv")

In [172]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,5 Series,2014,11200,Automatic,67068,Diesel,125,57.6,2.0
1,6 Series,2018,27000,Automatic,14827,Petrol,145,42.8,2.0
2,5 Series,2016,16000,Automatic,62794,Diesel,160,51.4,3.0
3,1 Series,2017,12750,Automatic,26676,Diesel,145,72.4,1.5
4,7 Series,2014,14500,Automatic,39554,Diesel,160,50.4,3.0


# Обучающая и тестовая выборка (train and test)

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
train, test = train_test_split(df, random_state=42, train_size = 0.6) # random_state чтобы эксперимент навсегда сохранился

# Валидационная выборка 

In [27]:
val, test = train_test_split(test, random_state=42, train_size = 0.5) # random_state чтобы эксперимент навсегда сохранился

In [28]:
len(test) / len(df) # train 60% val 20% test 20%

0.20007420461923756

# Список фичей 

In [ ]:
# признаки на которых будем обучать catboost

In [29]:
train.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [30]:
x = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax','mpg', 'engineSize'] 
# Фичи на которых мы обучаемся 

cat_features = ["model", "transmission", "fuelType"] # Список категориальных фичей 

y = ['price'] # target


In [31]:
train

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
5133,X6,2015,25000,Semi-Auto,26100,Diesel,205,45.6,3.0
2121,X2,2019,31980,Automatic,4804,Petrol,145,34.0,2.0
6576,X2,2018,22995,Semi-Auto,24136,Petrol,145,38.7,2.0
8330,3 Series,2016,12999,Manual,52224,Diesel,125,62.8,2.0
3161,X3,2019,32980,Semi-Auto,1961,Diesel,150,54.3,2.0
...,...,...,...,...,...,...,...,...,...
5734,3 Series,2018,17995,Semi-Auto,40456,Diesel,145,64.2,2.0
5191,1 Series,2016,19950,Semi-Auto,23210,Petrol,200,39.8,3.0
5390,X2,2019,25450,Semi-Auto,5652,Diesel,145,47.9,2.0
860,X1,2016,19850,Semi-Auto,14542,Diesel,125,60.1,2.0


# Принцип преобразования категориальных переменных 

In [41]:
train["transmission"]

5133    Semi-Auto
2121    Automatic
6576    Semi-Auto
8330       Manual
3161    Semi-Auto
          ...    
5734    Semi-Auto
5191    Semi-Auto
5390    Semi-Auto
860     Semi-Auto
7270    Automatic
Name: transmission, Length: 6468, dtype: object

In [40]:
pd.get_dummies(train["transmission"])

,Automatic,Manual,Semi-Auto
5133,False,False,True
2121,True,False,False
6576,False,False,True
8330,False,True,False
3161,False,False,True
...,...,...,...
5734,False,False,True
5191,False,False,True
5390,False,False,True
860,False,False,True


# Первый запуск Кэтбуста

In [44]:
train

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
5133,X6,2015,25000,Semi-Auto,26100,Diesel,205,45.6,3.0
2121,X2,2019,31980,Automatic,4804,Petrol,145,34.0,2.0
6576,X2,2018,22995,Semi-Auto,24136,Petrol,145,38.7,2.0
8330,3 Series,2016,12999,Manual,52224,Diesel,125,62.8,2.0
3161,X3,2019,32980,Semi-Auto,1961,Diesel,150,54.3,2.0
...,...,...,...,...,...,...,...,...,...
5734,3 Series,2018,17995,Semi-Auto,40456,Diesel,145,64.2,2.0
5191,1 Series,2016,19950,Semi-Auto,23210,Petrol,200,39.8,3.0
5390,X2,2019,25450,Semi-Auto,5652,Diesel,145,47.9,2.0
860,X1,2016,19850,Semi-Auto,14542,Diesel,125,60.1,2.0


In [45]:
from catboost import CatBoostRegressor

In [49]:
x = ['year', 'transmission', 'engineSize']

cat_features = [ "transmission"]

y = ['price']


In [136]:
model = CatBoostRegressor(cat_features=cat_features, 
                          eval_metric="MAPE",
                          random_seed=42,
                          early_stopping_rounds=200,
                          verbose=100) # Гипер параметры 

In [137]:
train[x]

,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
5133,X6,2015,Semi-Auto,26100,Diesel,205,45.6,3.0
2121,X2,2019,Automatic,4804,Petrol,145,34.0,2.0
6576,X2,2018,Semi-Auto,24136,Petrol,145,38.7,2.0
8330,3 Series,2016,Manual,52224,Diesel,125,62.8,2.0
3161,X3,2019,Semi-Auto,1961,Diesel,150,54.3,2.0
...,...,...,...,...,...,...,...,...
5734,3 Series,2018,Semi-Auto,40456,Diesel,145,64.2,2.0
5191,1 Series,2016,Semi-Auto,23210,Petrol,200,39.8,3.0
5390,X2,2019,Semi-Auto,5652,Diesel,145,47.9,2.0
860,X1,2016,Semi-Auto,14542,Diesel,125,60.1,2.0


In [138]:
model.fit(train[x], train[y],eval_set = (val[x], val[y])) # Запуск обучения

Learning rate set to 0.068263
0:	learn: 0.4607438	test: 0.4563750	best: 0.4563750 (0)	total: 6.25ms	remaining: 6.24s
100:	learn: 0.0904302	test: 0.0896388	best: 0.0896388 (100)	total: 234ms	remaining: 2.08s
200:	learn: 0.0781784	test: 0.0803125	best: 0.0803125 (200)	total: 441ms	remaining: 1.75s
300:	learn: 0.0729925	test: 0.0764506	best: 0.0764506 (300)	total: 668ms	remaining: 1.55s
400:	learn: 0.0695552	test: 0.0744676	best: 0.0744553 (399)	total: 892ms	remaining: 1.33s
500:	learn: 0.0670683	test: 0.0732827	best: 0.0732827 (500)	total: 1.12s	remaining: 1.12s
600:	learn: 0.0652835	test: 0.0726170	best: 0.0726128 (597)	total: 1.37s	remaining: 908ms
700:	learn: 0.0636320	test: 0.0720622	best: 0.0720500 (695)	total: 1.6s	remaining: 683ms
800:	learn: 0.0623408	test: 0.0714586	best: 0.0714329 (794)	total: 1.85s	remaining: 461ms
900:	learn: 0.0611487	test: 0.0711564	best: 0.0711321 (898)	total: 2.07s	remaining: 228ms
999:	learn: 0.0601407	test: 0.0708439	best: 0.0708403 (998)	total: 2.29s	r

In [139]:
model.predict(test[x])

array([57417.53972205, 22003.56027391, 13461.40876567, ...,
       30553.87789981, 13818.32099473, 24849.36058457])

In [83]:
test["price_pred"] = model.predict(test[x])

In [84]:
test

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,price_pred,price_pred_all
4984,X6,2019,52990,Semi-Auto,3086,Diesel,145,34.9,3.0,57417.539722,56918.762922
4880,X1,2018,24081,Semi-Auto,13245,Diesel,150,60.1,2.0,22003.560274,22437.019916
9132,1 Series,2016,12999,Automatic,68949,Petrol,200,43.5,2.0,13461.408766,13323.390597
6451,1 Series,2020,11995,Semi-Auto,10,Petrol,150,34.5,2.0,32149.002131,33416.841004
7199,3 Series,2020,29875,Semi-Auto,150,Petrol,145,42.2,2.0,35100.139852,34999.003042
...,...,...,...,...,...,...,...,...,...,...,...
9627,3 Series,2015,14999,Automatic,78680,Diesel,160,52.3,3.0,14271.263108,14283.884697
6096,4 Series,2019,29676,Automatic,7365,Petrol,145,41.5,3.0,31555.069740,30842.309886
1747,5 Series,2019,30570,Automatic,3067,Hybrid,145,49.6,2.0,30553.877900,30077.530287
4023,3 Series,2016,14999,Automatic,70054,Hybrid,0,148.7,2.0,13818.320995,13712.857110


In [85]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [86]:
def error(y_true, y_pred):
    print(mean_absolute_error(y_true, y_pred))
    print(mean_absolute_percentage_error(y_true, y_pred))

In [87]:
error(test["price"], test["price_pred"])

1556.5378327680894
0.07367840910148211


# Обучаемся на всех фичах 

In [88]:
# Все фичи 

x = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax','mpg', 'engineSize']

cat_features = ["model", "transmission", "fuelType"]

y = ['price']


In [95]:
parametrs = {"cat_features":cat_features,
            "eval_metric": "MAPE",
            "random_seed":42,
            "verbose": 100}

In [96]:
model = CatBoostRegressor(**parametrs)

In [97]:
model.fit(train[x], train[y], eval_set=(val[x], val[y]))

Learning rate set to 0.068263
0:	learn: 0.4607438	test: 0.4563750	best: 0.4563750 (0)	total: 6.75ms	remaining: 6.75s
100:	learn: 0.0904302	test: 0.0896388	best: 0.0896388 (100)	total: 235ms	remaining: 2.1s
200:	learn: 0.0781784	test: 0.0803125	best: 0.0803125 (200)	total: 449ms	remaining: 1.79s
300:	learn: 0.0729925	test: 0.0764506	best: 0.0764506 (300)	total: 678ms	remaining: 1.57s
400:	learn: 0.0695552	test: 0.0744676	best: 0.0744553 (399)	total: 895ms	remaining: 1.34s
500:	learn: 0.0670683	test: 0.0732827	best: 0.0732827 (500)	total: 1.12s	remaining: 1.12s
600:	learn: 0.0652835	test: 0.0726170	best: 0.0726128 (597)	total: 1.34s	remaining: 892ms
700:	learn: 0.0636320	test: 0.0720622	best: 0.0720500 (695)	total: 1.56s	remaining: 665ms
800:	learn: 0.0623408	test: 0.0714586	best: 0.0714329 (794)	total: 1.77s	remaining: 440ms
900:	learn: 0.0611487	test: 0.0711564	best: 0.0711321 (898)	total: 2s	remaining: 220ms
999:	learn: 0.0601407	test: 0.0708439	best: 0.0708403 (998)	total: 2.23s	rema

In [98]:
test["price_pred_all"] = model.predict(test[x])

In [99]:
error(test["price"], test["price_pred_all"]) # Алгоритм (Модель) предсказывет цену с вероятностью ошибки 7% 

1556.5378327680894
0.07367840910148211


# Количесво иттераций и learning ratw 

In [103]:
x = ['year', 'transmission', 'engineSize']

cat_features = [ "transmission"]

y = ['price']


In [116]:
parametrs = {"cat_features":cat_features,
            "eval_metric": "MAPE",
            "learning_rate" : 0.03, # Темп работы 
            "random_seed":42,
            "verbose": 100}

In [117]:
model = CatBoostRegressor(**parametrs)

In [118]:
model.fit(train[x], train[y], eval_set=(val[x], val[y]))

0:	learn: 0.4746987	test: 0.4699730	best: 0.4699730 (0)	total: 4.81ms	remaining: 4.81s
100:	learn: 0.1681230	test: 0.1638970	best: 0.1638970 (100)	total: 177ms	remaining: 1.57s
200:	learn: 0.1561398	test: 0.1538311	best: 0.1538311 (200)	total: 309ms	remaining: 1.23s
300:	learn: 0.1543096	test: 0.1525317	best: 0.1525309 (299)	total: 422ms	remaining: 980ms
400:	learn: 0.1534465	test: 0.1519221	best: 0.1519198 (397)	total: 534ms	remaining: 797ms
500:	learn: 0.1527824	test: 0.1516330	best: 0.1516164 (480)	total: 663ms	remaining: 660ms
600:	learn: 0.1522590	test: 0.1514284	best: 0.1514284 (600)	total: 795ms	remaining: 528ms
700:	learn: 0.1518052	test: 0.1513704	best: 0.1513673 (689)	total: 917ms	remaining: 391ms
800:	learn: 0.1514495	test: 0.1513967	best: 0.1513651 (703)	total: 1.04s	remaining: 260ms
900:	learn: 0.1510958	test: 0.1513657	best: 0.1513489 (873)	total: 1.18s	remaining: 130ms
999:	learn: 0.1508297	test: 0.1513297	best: 0.1513168 (992)	total: 1.31s	remaining: 0us

bestTest = 0.1

In [119]:
# Best Test - точка переобучения 

In [120]:
# Все фичи 

x = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax','mpg', 'engineSize']

cat_features = ["model", "transmission", "fuelType"]

y = ['price']


In [133]:
parametrs = {"cat_features":cat_features,
            "early_stopping_rounds": 300, # Сокращает нам время всего 200
            "eval_metric": "MAPE",
            "random_seed":42,
             "learning_rate": 0.08,
            "verbose": 100}

In [134]:
model = CatBoostRegressor(**parametrs)

In [135]:
model.fit(train[x], train[y], eval_set=(val[x], val[y]))

0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 5.98ms	remaining: 5.97s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 237ms	remaining: 2.11s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 446ms	remaining: 1.77s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 668ms	remaining: 1.55s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 888ms	remaining: 1.32s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 1.12s	remaining: 1.12s
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 1.38s	remaining: 916ms
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 1.62s	remaining: 692ms
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 1.84s	remaining: 459ms
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 2.07s	remaining: 227ms
999:	learn: 0.0584626	test: 0.0686829	best: 0.0686377 (993)	total: 2.29s	remaining: 0us

bestTest = 0.0

# Обучение на всех данных 

In [140]:
len(train)

6468

In [141]:
len(val)

2156

In [143]:
train_full = pd.concat([train, val])

In [144]:
model.fit(train[x], train[y], eval_set=(val[x], val[y]))

Learning rate set to 0.068263
0:	learn: 0.4607438	test: 0.4563750	best: 0.4563750 (0)	total: 6.76ms	remaining: 6.75s
100:	learn: 0.0904302	test: 0.0896388	best: 0.0896388 (100)	total: 228ms	remaining: 2.03s
200:	learn: 0.0781784	test: 0.0803125	best: 0.0803125 (200)	total: 436ms	remaining: 1.73s
300:	learn: 0.0729925	test: 0.0764506	best: 0.0764506 (300)	total: 659ms	remaining: 1.53s
400:	learn: 0.0695552	test: 0.0744676	best: 0.0744553 (399)	total: 871ms	remaining: 1.3s
500:	learn: 0.0670683	test: 0.0732827	best: 0.0732827 (500)	total: 1.09s	remaining: 1.09s
600:	learn: 0.0652835	test: 0.0726170	best: 0.0726128 (597)	total: 1.32s	remaining: 878ms
700:	learn: 0.0636320	test: 0.0720622	best: 0.0720500 (695)	total: 1.55s	remaining: 663ms
800:	learn: 0.0623408	test: 0.0714586	best: 0.0714329 (794)	total: 1.79s	remaining: 445ms
900:	learn: 0.0611487	test: 0.0711564	best: 0.0711321 (898)	total: 2.05s	remaining: 225ms
999:	learn: 0.0601407	test: 0.0708439	best: 0.0708403 (998)	total: 2.31s	r

In [145]:
model.best_iteration_

998

In [147]:
parametrs = {"iterations" : model.best_iteration_ + 1,
             "cat_features":cat_features,
            "eval_metric": "MAPE",
            "random_seed":42,
             "learning_rate": 0.08,
            "verbose": 100}

In [148]:
model = CatBoostRegressor(**parametrs)

In [149]:
model.fit(train[x], train[y], eval_set=(val[x], val[y]))

0:	learn: 0.4564416	test: 0.4521294	best: 0.4521294 (0)	total: 6.39ms	remaining: 6.38s
100:	learn: 0.0875290	test: 0.0867329	best: 0.0867329 (100)	total: 238ms	remaining: 2.11s
200:	learn: 0.0760834	test: 0.0779235	best: 0.0779235 (200)	total: 443ms	remaining: 1.76s
300:	learn: 0.0708670	test: 0.0740290	best: 0.0740290 (300)	total: 668ms	remaining: 1.55s
400:	learn: 0.0676240	test: 0.0718814	best: 0.0718814 (400)	total: 889ms	remaining: 1.32s
500:	learn: 0.0654070	test: 0.0709221	best: 0.0708594 (496)	total: 1.13s	remaining: 1.12s
600:	learn: 0.0638577	test: 0.0704606	best: 0.0704490 (599)	total: 1.39s	remaining: 920ms
700:	learn: 0.0622969	test: 0.0698226	best: 0.0698226 (700)	total: 1.62s	remaining: 690ms
800:	learn: 0.0609583	test: 0.0694169	best: 0.0694004 (791)	total: 1.85s	remaining: 458ms
900:	learn: 0.0596089	test: 0.0689968	best: 0.0689945 (895)	total: 2.07s	remaining: 225ms
998:	learn: 0.0584760	test: 0.0686821	best: 0.0686377 (993)	total: 2.29s	remaining: 0us

bestTest = 0.0

In [150]:
parametrs = {"cat_features":cat_features,
            "early_stopping_rounds": 300, # Сокращает нам время всего 200
            "eval_metric": "MAPE",
            "random_seed":42,
             "learning_rate": 0.08,
            "verbose": 100}

In [151]:
modl = CatBoostRegressor(**parametrs)

In [152]:
model.fit(train_full[x], train_full[y])

0:	learn: 0.4538016	total: 5.97ms	remaining: 5.96s
100:	learn: 0.0854178	total: 262ms	remaining: 2.33s
200:	learn: 0.0745022	total: 511ms	remaining: 2.03s
300:	learn: 0.0701717	total: 758ms	remaining: 1.76s
400:	learn: 0.0676577	total: 992ms	remaining: 1.48s
500:	learn: 0.0656926	total: 1.24s	remaining: 1.23s
600:	learn: 0.0638104	total: 1.48s	remaining: 982ms
700:	learn: 0.0624302	total: 1.74s	remaining: 741ms
800:	learn: 0.0612275	total: 2.01s	remaining: 496ms
900:	learn: 0.0599823	total: 2.26s	remaining: 246ms
998:	learn: 0.0589834	total: 2.5s	remaining: 0us


In [153]:
model.predict(test[x])

array([56425.93560989, 22685.27100567, 13453.70567237, ...,
       30223.8490056 , 13790.96765559, 24777.4632018 ])

In [154]:
test["price_pred_all_features_and_data"] = model.predict(test[x])

In [155]:
error(test["price"], test["price_pred_all_features_and_data"])

1512.4017819023943
0.07305915524950393


# Метрики регрессиии 

link https://catboost.ai/en/docs/concepts/loss-functions

# Меняем функцию оптимизации 

In [157]:
# Все фичи 

x = ['model', 'year', 'transmission', 'mileage', 'fuelType', 'tax','mpg', 'engineSize']

cat_features = ["model", "transmission", "fuelType"]

y = ['price']


In [164]:
parametrs = {"cat_features":cat_features,
            "eval_metric": "MAPE",
             
            "loss_function": "MAE", # Меняем функцию оптимизации  
    
            "random_seed":42,
             "learning_rate": 0.1,
            "verbose": 100}

In [165]:
model = CatBoostRegressor(**parametrs)

In [171]:
model.fit(train_full[x], train_full[y], eval_set=(val[x], val[y])) # Сам процесс обучения 

0:	learn: 0.3820228	test: 0.3774045	best: 0.3774045 (0)	total: 9.13ms	remaining: 9.12s
100:	learn: 0.0759335	test: 0.0729437	best: 0.0729437 (100)	total: 330ms	remaining: 2.93s
200:	learn: 0.0664370	test: 0.0647293	best: 0.0647293 (200)	total: 646ms	remaining: 2.57s
300:	learn: 0.0628444	test: 0.0616085	best: 0.0616085 (300)	total: 957ms	remaining: 2.22s
400:	learn: 0.0604654	test: 0.0596538	best: 0.0596538 (400)	total: 1.34s	remaining: 2s
500:	learn: 0.0587112	test: 0.0583472	best: 0.0583472 (500)	total: 1.65s	remaining: 1.65s
600:	learn: 0.0575951	test: 0.0575754	best: 0.0575695 (599)	total: 1.96s	remaining: 1.3s
700:	learn: 0.0566398	test: 0.0569207	best: 0.0569207 (700)	total: 2.26s	remaining: 963ms
800:	learn: 0.0558118	test: 0.0563467	best: 0.0563411 (795)	total: 2.56s	remaining: 636ms
900:	learn: 0.0551435	test: 0.0558035	best: 0.0558035 (900)	total: 2.86s	remaining: 314ms
999:	learn: 0.0544953	test: 0.0553170	best: 0.0553170 (999)	total: 3.15s	remaining: 0us

bestTest = 0.05531